# Seqs Luis

In [ ]:
class Mat:
    """
    Classe que permite criar e manipular Matrizes
   
    """
    def __init__(self, rows, cols):
        """Construtor da Matriz 

        Args:
            rows (int): Número de linhas
            cols (int): Número de colunas 
        """
        self.mat = [[0 for c in range(cols)]
                    for r in range(rows)]
    

    def numRows (self): 
        """Função que permite obter o número de linhas da Matriz

        Returns:
            int : Valor de linhas da Matriz
        """
        return len(self.mat)


    def numCols (self): 
        """Função que permite obter o númeor de colunas da Matriz

        Returns:
            int : Valor de colunas da Matriz
        """
        return len(self.mat[0])


    def __str__(self):
        """ Função que permite devolver a Matriz como uma string

        Returns:
            str : String da Matriz
        """
        return '\n'.join(' '.join(str(val) for val in row)
                         for row in self.mat)

    
    def __getitem__ (self, n):
        return self.mat[n]

In [ ]:
import io

def subst(x, y):
  import blosum as bl

  """Funçao que permite obter o valor da substituição do Aminoácido da Blossum 62

  Args:
      x (str): Aminoácido 1
      y (str): Aminoácido 2

  Returns:
      int: Valor de substituição do Aminoácido 
  """
  
  dic = dict(bl.BLOSUM(62))
  return int(dic[x+y])   # feito


In [ ]:
class NW:
  """Classe que permite efetuar o Algoritmo de Needleman Wunsch de forma a realizar o alinhamento Global de um par de sequências.
  """

  def __init__(self, s1, s2, g = -4):
    """Contrutor da classe NW

    Args:
        s1 (str): Sequência 1
        s2 (str): Sequência 2
        g (int, optional): Gap Penalty. Defaults to -4.
    """
    self.s1 = s1
    self.s2 = s2
    self.mat = Mat(len(s1) + 1, len(s2) + 1)  
    self.tr  = Mat(len(s1) + 1, len(s2) + 1)
    
    for L in range(len(s1)):                     
      self.mat[L + 1][0] = g * (L + 1)           
      self.tr[L + 1][0]  = 'C'                      

    for C in range(len(s2)):                     
      self.mat[0][C + 1] = g * (C + 1)             
      self.tr[0][C + 1]  = 'E'                        

    for L, x1 in enumerate(s1):
      for C, x2 in enumerate(s2):
        possiveis = [
            self.mat[L  ][C    ] + subst(x1, x2),   
            self.mat[L+1][C    ] + g,               
            self.mat[L  ][C + 1] + g,               
        ]
        dirs = "DEC"

        self.mat[L + 1][C + 1] = max(possiveis)   
        self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]  

  def rebuild(self):
    """Função que permite obter um possivel alinhamento das sequências

    Returns:
        str: Sequências alinhadas 
    """
    L = len(self.s1)
    C = len(self.s2)
    S1 = ""
    S2 = ""
    
    dirs = {
        'D' : (-1, -1),
        'E' : ( 0, -1),
        'C' : (-1,  0)
    }

    while L > 0 or C > 0:
      # print(self.s1[L - 1], self.s2[C - 1])
      # print(L, C)
      # print(self.tr[L][C])
      DL, DC = dirs[self.tr[L][C]]

      if self.tr[L][C] == "D":
        S1 = self.s1[L - 1] + S1    # este self.s1/s2 representa as sequencias 
        S2 = self.s2[C - 1] + S2
      elif self.tr[L][C] == "E":
        S1 = '-' + S1
        S2 = self.s2[C - 1] + S2
      else:
        S1 = self.s1[L - 1] + S1
        S2 = '-' + S2        
      #print(S1, S2)
      L += DL
      C += DC

    return S1, S2
    # return print(S1, S2 , sep="\n")
 
  def max_score (self):
    """Função que permite obter o max score do alinhamento 

    Returns:
        int: Max Score
    """
    return self.mat[len(self.s1)][len(self.s2)]

  def __repr__(self):
    """Função que permite obter a representação em String da própria classe 

    Returns:
        str: Matriz S (Score) e Matriz T (Trace)
    """
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print(' ', *cols, sep = '   ', file = S)           
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print(file = S)

      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return S.getvalue()

  def score_mat(self):
    """Função que permite obter a Matriz S (Score)

    Returns:
        str: Matriz S
    """
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print(' ', *cols, sep = '   ', file = S)           
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print(file = S)
      return print(S.getvalue())

  def trace_mat(self):
    """Função que permite obter a Matriz T (Trace)

    Returns:
        str: Matriz T (Trace)
    """
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return print(S.getvalue())


In [ ]:
import unittest

class Test_NW(unittest.TestCase):
    
    
    def test_rebuild (self):

        self.assertEqual(NW("ACTG", "ACTG").rebuild(), ('ACTG', 'ACTG'))
        self.assertEqual(NW("ATCAT", "ACAT").rebuild(), ('ATCAT', 'A-CAT'))


    def test_max_score (self):

        self.assertEqual(NW("ACTG", "ACTG").max_score(), 24)
        self.assertEqual(NW("ATCAT", "ACAT").max_score(), 18)



unittest.main(argv=[''], exit=False)

In [ ]:
class SW:
  """Classe que permite efetuar o Algoritmo de Smith Waterman de forma a realizar o alinhamento Local de um par de sequências.
  """


  def __init__(self, s1, s2, g = -4):
    """Contrutor da classe SW

    Args:
        s1 (str): Sequência 1
        s2 (str): Sequência 2
        g (int, optional): Gap Penalty. Defaults to -4.
    """

    self.s1 = s1
    self.s2 = s2
    self.mat = Mat(len(s1) + 1, len(s2) + 1)  # este + 1 por causa dos "cabeçalhos" com as seqs
    self.tr  = Mat(len(s1) + 1, len(s2) + 1)


    for L, x1 in enumerate(s1):
      for C, x2 in enumerate(s2):
        possiveis = [
            self.mat[L  ][C    ] + subst(x1, x2),   # Diagonal
            self.mat[L+1][C    ] + g,               # Esquerda
            self.mat[L  ][C + 1] + g,               # Cima
            0]
        dirs = "DEC."

        self.mat[L + 1][C + 1] = max(possiveis)   #colocar na matriz o valor max das 3 possibilidades
        if self.mat[L + 1][C + 1] != 0:
          self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]  # colocar D E C dependendo da posiçao onde foi obtido o max (self,mat[L +1][C + 1])
  
  def max_score (self):
    """Função que permite obter o max score do alinhamento 

    Returns:
        int: Valor do Max Score
    """
    max_score = 0
    for L, x1 in enumerate(self.mat):
      for C, x2 in enumerate(self.mat):
        if max(x1) > max_score:
          max_score = max(x1)
    return max_score

  
  def rebuild(self):
    """Função que permite obter um possivel alinhamento das sequências

    Returns:
        str: Sequências alinhadas 
    """
    max_score = 0
    linha = 0
    coluna = 0
    for L, x1 in enumerate(self.mat):
      for C, x2 in enumerate(self.mat):
        if max(x1) > max_score:
          max_score = max(x1)
          linha = L
        coluna = C
    coluna -= 1
    L = linha
    C = coluna + 1
    S1 = ""
    S2 = ""
    
    dirs = {
        'D' : (-1, -1),
        'E' : ( 0, -1),
        'C' : (-1,  0),
        "." : (0, 0)
    }

    while L >= 0 or C >= 0:
      # print(self.s1[L - 1], self.s2[C - 1])
      # print(L, C)
      # print(self.tr[L][C])
      try:
        DL, DC = dirs[self.tr[L][C]]
        if self.tr[L][C] == "D":
            S1 = self.s1[L - 1] + S1    # este self.s1/s2 representa as sequencias 
            S2 = self.s2[C - 1] + S2
        elif self.tr[L][C] == "E":
            S1 = '-' + S1
            S2 = self.s2[C - 1] + S2
        elif self.tr[L][C] == "C":
            S1 = self.s1[L - 1] + S1
            S2 = '-' + S2 
        else:
          break
        # print(S1)
        # print(S2)     
        L += DL
        C += DC
      except:
        break

    
    return S1, S2
    # return print(S1, S2 , sep="\n")
 

        

    

  def __repr__(self):
    """Função que permite obter a representação em String da própria classe 

    Returns:
        str: Matriz S (Score) e Matriz T (Trace)
    """
    
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print(' ', *cols, sep = '   ', file = S)           # Kinda COmplicado perceber mas isto permite fazer print das 2 matrizes Valores e Trace
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print(file = S)

      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return S.getvalue()

  def score_mat(self):
    """Função que permite obter a Matriz S (Score)

    Returns:
        str: Matriz S
    """
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print(' ', *cols, sep = '   ', file = S)           # Kinda COmplicado perceber mas isto permite fazer print das 2 matrizes Valores e Trace
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print(file = S)
      return print(S.getvalue())

  def trace_mat(self):
    """Função que permite obter a Matriz T (Trace)

    Returns:
        str: Matriz T (Trace)
    """
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return print(S.getvalue())

In [ ]:
import unittest

class Test_SW(unittest.TestCase):
    
    
    def test_rebuild (self):

        self.assertEqual(SW("ACTG", "ACTG").rebuild(), ('ACTG', 'ACTG'))
        self.assertEqual(SW("ATCATAA", "ATTACAT").rebuild(), ('T-CAT', 'TACAT'))


    def test_max_score (self):

        self.assertEqual(SW("ACTG", "ACTG").max_score(), 24)
        self.assertEqual(SW("ATCATAA", "ATTACAT").max_score(), 19)

        
unittest.main(argv=[''], exit=False)

In [ ]:
def consensus(s1, s2):
    """Função que permite obter o Consensus das 2 Sequências 

    Args:
        s1 (str): Sequencia 1
        s2 (str): Sequencia 2

    Returns:
        str: Consensus das Sequências
    """
    res = ""
    for x, y in zip(s1, s2):
        if x == y:
            res += x
        elif x == '-':
            res += y
        else:
            res += x
    return res
    
class MultipleAlign:
    """Classe que permite realizar o alinhamento múltiplo de várias sequências 
    """
    def __init__(self, lista_seqs):
        """Contrutor da classe MultpleAlign

        Args:
            lista_seqs (list): Lista de sequências 
        """
        self.lista_seqs = lista_seqs
        self.len = len(lista_seqs)



    def __len__ (self):
        """Função que permite implementar a função len()

        Returns:
            int: Numero de Sequências presentes na Classe
        """

        return len(self.lista_seqs)
    

    def __repr__(self):
        """Função que permite representar a Classe como String 

        Returns:
            str: Diferentes Sequências presentes na Classe
        """

        return str(self.lista_seqs)
    
    def __getitem__(self,a):
        """Função que permite implementar a Indexação na Classe

        Args:
            a (int): Index

        Returns:
            str: Sequência correspondente ao Index 
        """
        return self.lista_seqs[a]
    
    
    def align (self):
        """Função que permite realizar o Alinhamento Progressivo das Sequências 

        Returns:
            list: Lista com o alinhamento entre todas as sequências
        """
        lista_final_con = self.lista_seqs.copy()
        lista_final = []
        c = 0
        while c < len(self.lista_seqs)-1:
            lista_temp = lista_final_con[0:2].copy()
            a = NW(lista_temp[0],lista_temp[1]).rebuild()
            con = consensus(a[0],a[1])
            del lista_final_con[0:2]
            lista_final_con.insert(0,con)
            c += 1

        i = len(self.lista_seqs)

        for c in range(i):
            # print(NW(self.lista_seqs[c],lista_final_con[0]).rebuild()[0])
            lista_final.append(NW(self.lista_seqs[c],lista_final_con[0]).rebuild()[0])
        

        return lista_final

In [ ]:
class Test_MultipleAlign (unittest.TestCase):

    def test_align (self):
        self.assertEqual(MultipleAlign(["ATAGC", "AACC", "ATGAC","ATAGC", "AACC", "ATGAC"]).align(), ['ATAGC', 'A-ACC', 'ATGAC', 'ATAGC', 'A-ACC', 'ATGAC'])
        self.assertEqual(MultipleAlign(["TACCCGCT","AACCA"]).align(), ['TACCCGCT', '-A-AC-CA'])


unittest.main(argv=[''], exit=False)

In [ ]:
import mysql.connector as SQLC  

DataBase = SQLC.connect(
   host ="127.0.0.1",
   user ="root",
   password ="login123",
   database = "Base Dados IAP",
   auth_plugin="mysql_native_password"
)
DataBase.autocommit= True
cur = DataBase.cursor()

In [ ]:
def connect_database (host ="127.0.0.1", user ="root",password ="login123", database = "Base Dados IAP"):
    """Funçao que permite ao utilizador conectar à base de dados (Base Dados IAP)

    Args:
        host (str, optional):  Host . Defaults to "127.0.0.1".
        user (str, optional):  User . Defaults to "root".
        password (str, optional):  Password . Defaults to "login123".
        database (str, optional):  Database . Defaults to "Base Dados IAP".
    """

    DataBase = SQLC.connect(

        host = host,
        user = user,
        password = password,
        database = database,
        auth_plugin="mysql_native_password"
    )
    DataBase.autocommit= True
    cur = DataBase.cursor()

def get_all_organisms ():
    """Função que permite obter todos os organismos presentes na Base de Dados Predefinida

    Returns:
        lista: Contêm nome de Organimos 
    """
    connect_database()
    cur.execute("""select Organism from Genbank""")
    organisms_DB = cur.fetchall()

    return organisms_DB


def get_seq_from_organism (organism):
    """Função que permite obter a sequência do Organismo pretendido

    Args:
        organism (str): Organismo Presente na base de dados 

    Returns:
        str: Sequência do Organismo
    """
    connect_database()
    cur.execute(f""" select Sequence from Sequences
                    join Genbank on Sequences.ID_version_seq = Genbank.ID_version_seq
                    where Organism = '{organism}'
    """)

    sequence = cur.fetchall()
    
    return sequence[0][0]

def get_all_seqs():
    """Função que permite obter todas as sequências presentes na Base de Dados

    Returns:
        list: Lista com todas sequências presentes na Base de Dados
    """
    connect_database()
    cur.execute(f""" select Sequence from Sequences
    """)

    sequence = cur.fetchall()

    return sequence



In [ ]:
import unittest

class Test_DataBase (unittest.TestCase):
    def test_database (self):
        self.assertEqual(len(get_all_organisms()),len(get_all_seqs()))


    def test_database_2 (self):
        self.assertEqual(type(get_seq_from_organism('Sulfolobus spindle-shaped virus 1')), str)


unittest.main(argv=[''], exit=False)

In [ ]:
class Blast ():
    """Classe que permite realizar um Blast Simplificado 
    """
    def __init__(self, query, seq, w = 3):
        """Construtor do Blast Simplificado 

        Args:
            query (str): Query 
            seq (str): Seq
            w (int, optional): W . Defaults to 3.
            
        """
        self.seq = seq
        self.query = query
        self.w = w

    def __len__(self):
        return len(self.query)

    
    def query_map(query, w):
        """Função que permite devolver um dicionário em que as chaves são as sequências e os valores são uma lista dos índices

        Args:
            query (str): Query
            w (int): W

        Returns:
            dict: Dicionário em que as chaves são as sequências e os valores são uma lista dos índice
        """
        
        tam = len(query)
        res = {}
        for chave, offset in [(query[p : p + w], p) for p in range(0, tam - w +1)]:
            if chave not in res:
                res[chave] = []
            res[chave].append(offset)
        return(res)


    def get_all_offsets(s1, s2):  # s1 --> Padrão    s2 --> seq_analisar
        """Função Auxiliar que permite devolver todos os índices das ocorrências de uma substring numa string

        Args:
            s1 (str): Substring
            s2 (str): String

        Returns:
            dict: Dicionário com todas as ocorrèncias da substring na string
        """
        w = len(s1)

        return[p for p in range(0, len(s2) - w + 1) if s2[p : p + w] == s1]


    def hits(qm, seq):
        """Função que permite gerar uma lista de hits

        Args:
            qm (dict): Dicionário proveninete da Função query_map
            seq (str): Sequência Proveniente da base de dados
        
        Returns:
            list: Lista de hits em que cada elemento é um tuplo com os indices
        """
              
        return[(o_query, o_seq) for chave, offsets in qm.items() 
        for o_query in offsets 
        for o_seq in Blast.get_all_offsets(chave, seq)]


    def extend_hit_any_direction(query, seq, o1, o2, dir):
        """Função auxliar que permite extender o hit em ambas as direções

        """
        matches = 0
        count = 0
        while o1 >= 0 and o2 >= 0 and o1 < len(query) and o2 < len(seq):
            matches += 1 if query[o1] == seq[o2] else 0
            count += 1
            if 2 * matches < count:
                return o1, o2, matches, count
            o1 += dir
            o2 += dir
        return o1 - dir, o2 - dir, matches, count


    def extend_hit(query, seq, hit, w):
        """Função que permite estende um hit em cada direção se o nº de matches for de pelo menos metade do tamanho da extensão

        Args:
            query (str): Query
            seq (str): Seq
            hit (tuple): Hit
            w (int): w

        Returns:
            tuple: Tuplo com o índice do início do hit estendido na query, na sequência, o tamanho e o nºde matches
        """

        o1, o2 = hit
        left  = Blast.extend_hit_any_direction(query, seq, o1 - 1, o2 - 1, -1)
        right = Blast.extend_hit_any_direction(query, seq, o1 + w, o2 + w, +1)

        O1, O2, ML, SL = left
        _,   _, MR, SR = right
        return O1, O2, w + SL + SR, ML + w + MR
    

    def best_hit (self):
        """Função que permite devolver a extensão de maior score (no caso de empate, deverá devolver a de menor tamanho que aparece primeiro)

        Returns:
            tuple: Tuplo com a extensão que possui maior score
        """
        qm = Blast.query_map(self.query,self.w)
        lista_hits = Blast.hits(qm, self.seq)
        all_scores =[Blast.extend_hit(self.query, self.seq, c, self.w) for c in lista_hits]
        all_scores.sort(key=lambda c: c[2] and c[3], reverse=True)
        if all_scores == []:
            return print("Not Found")
        else:
            return all_scores [0]

In [ ]:
import unittest

class Test_Blast (unittest.TestCase):
    """Estes Testes foram baseados nos exemplos fornecidos pelo Professor Rui (inputs/outputs) durantes
    as aulas teóricas 
    """
    query = "AATATAT"
    seq = "AATATGTTATATAATAATATTT"
    w = 3

    def test_query_map (self):
        self.assertEqual(Blast.query_map(query,w),{'AAT': [0], 'ATA': [1, 3], 'TAT': [2, 4]})
    

    def test_get_all_offsets (self):
        self.assertEqual(Blast.get_all_offsets(query,seq), [])
    
    
    def test_hits (self):
        self.assertEqual(Blast.hits(Blast.query_map(query,w),seq),[
            (0, 0), (0, 12), (0, 15), (1, 1), (1, 8),
            (1, 10), (1, 13), (1, 16), (3, 1), (3, 8), (3, 10),
            (3, 13), (3, 16), (2, 2), (2, 7), (2, 9), (2, 17),
            (4, 2), (4, 7), (4, 9),(4, 17)])


    def test_extend_hit_any_direction_left (self):
        self.assertEqual(Blast.extend_hit_any_direction(query, seq, 1, 16, -1),(0, 15, 2, 2))

    
    def test_extend_hit_any_direction_right (self):
        self.assertEqual(Blast.extend_hit_any_direction(query, seq, 1, 16, 1),(6, 21, 5, 6))

    def test_extend_hit (self):
        self.assertEqual(Blast.extend_hit(query, seq, (1, 16), w), (0, 15, 7, 6))

    
    def test_best_hit (self):
        self.assertEqual(Blast(query, seq, w).best_hit(), (0, 0, 7, 6))

unittest.main(argv=[''], exit=False)

# Luis


In [ ]:
class Motifs:
    """Classe que permite gerar Motifs probabilisticos 
    """
    def __init__(self, list_seqs):
        """Contrutor do Motifs onde criámos o nosso Alfabeto(letters) e onde gerámos a nossa Matriz vazia (mat)

        Args:
            list_seqs (list): Lista contendo diferentes sequências biológicas
        """
        from collections import Counter
        self.seqs = list_seqs
        self.letters = list(Counter([a for c in list_seqs for a in c]))
        self.mat = Mat(len(self.letters)+1,len(list_seqs[0])+1)
    
    def __repr__(self):
        return print(self.mat)

    def __str__(self) -> str:
        return print(self.mat)

    def contagens (self):
        """Função que permite devolver a frequência dos elementos das sequências

        Returns:
            list: Matriz das Contagens 
        """
        for c in self.seqs:
            for coluna in range(len(self.seqs[0])):
                linha = self.letters.index(c[coluna])
                self.mat[linha + 1][coluna + 1] += 1
                self.mat[0][coluna + 1] = coluna +1 # cabeçalho
                self.mat[0][0] = "."                # same
                self.mat[linha +1][0] = self.letters[linha] #same
        return print(self.mat)


    def PWM (self, pseudo_count = 0):
        """Função que permite gerar a Matriz PWM

        Args:
            pseudo_count (int, optional): pseudocontagem. Defaults to 0.

        Returns:
            list: Matriz PWM
        """
        self.mat = Mat(len(self.letters)+1,len(self.seqs[0])+1)
        for c in self.seqs:
            for coluna in range(len(self.seqs[0])):
                linha = self.letters.index(c[coluna])
                self.mat[linha + 1][coluna + 1] += 1
                self.mat[0][coluna + 1] = f"  {coluna +1} "   " "  # cabeçalho
                self.mat[0][0] = "."                # same
                self.mat[linha +1][0] = self.letters[linha] #same
        try:      
            for c in range(1,len(self.letters)+ 1):
                for a in range(1,len(self.seqs)+ 1): #-----------------------------------------------
                    self.mat[c][a] = round(float(self.mat[c][a] + pseudo_count + 0.01) / float(len(self.seqs)),3)
            return print(self.mat)

        except:
            for c in range(1,len(self.letters)+ 1):
                for a in range(1,len(self.seqs)+ 2): #-----------------------------------------------
                    self.mat[c][a] = round(float(self.mat[c][a] + pseudo_count + 0.01) / float(len(self.seqs)),3)
            return print(self.mat)

    
    def PSSM (self, pseudo_count = 0):
        """Função que permite gerar a Matriz PSSM

        Args:
            pseudo_count (int, optional): pseudocontagem. Defaults to 0.

        Returns:
            list: Matriz PSSM
        """
        self.mat = Mat(len(self.letters)+1,len(self.seqs[0])+1)
        from math import log2
        for c in self.seqs:
            for coluna in range(len(self.seqs[0])):
                linha = self.letters.index(c[coluna])
                self.mat[linha + 1][coluna + 1] += 1
                self.mat[0][coluna + 1] = f"  {coluna +1} "   " "  # cabeçalho
                self.mat[0][0] = "."                # same
                self.mat[linha +1][0] = self.letters[linha] #same
        try:
            for c in range(1,len(self.letters)+ 1):
                for a in range(1,len(self.seqs)+ 1): #--------------------------------
                    self.mat[c][a] = round((log2(((self.mat[c][a]+ pseudo_count)/(len(self.seqs)+ len(self.letters)* pseudo_count)/(1/len(self.letters)) ))),3)
            return print(self.mat)
        except:
            for c in range(1,len(self.letters)+ 1):
                for a in range(1,len(self.seqs)+ 2): #--------------------------------
                    self.mat[c][a] = round((log2(((self.mat[c][a]+ pseudo_count)/(len(self.seqs)+ len(self.letters* pseudo_count)/(1/len(self.letters)) )))),3)
            return print(self.mat)
    
    def prob_seq(self, seq):
        """Função que permite devolver a probabilidade de uma sequência

        Args:
            seq (str): Sequência 

        Returns:
            float: Probabilidade da sequência 
        """

        self.mat = Mat(len(self.letters)+1,len(self.seqs[0])+1)
        prob = 1 
        for c in self.seqs:
            for coluna in range(len(self.seqs[0])):
                linha = self.letters.index(c[coluna])
                self.mat[linha + 1][coluna + 1] += 1
                self.mat[0][coluna + 1] = f"  {coluna +1} "   " "  # cabeçalho
                self.mat[0][0] = "."                # same
                self.mat[linha +1][0] = self.letters[linha] #same
        for c in range(1,len(self.letters)+ 1):
            for a in range(1,len(self.seqs)+ 1):
                self.mat[c][a] = round(float(self.mat[c][a] + 0.01) / float(len(self.seqs)),3)
        
        for c in range(1,len(self.seqs[0])+ 1):
            a = self.letters.index(seq[c - 1])
            prob *= self.mat[a +1][c]
        return round(prob,4)


    def most_prob_seq (self,pseudo_count = 0):
        """Função que permite devolver a sequência mais provável de ocorrer

        Args:
            pseudo_count (int, optional): pseudocontagem. Defaults to 0.

        Returns:
            str: Sequência mais provável de ocorrer
        """

        self.mat = Mat(len(self.letters)+1,len(self.seqs[0])+1)
        resul = ""
        for c in self.seqs:
            for coluna in range(len(self.seqs[0])):
                linha = self.letters.index(c[coluna])
                self.mat[linha + 1][coluna + 1] += 1
                self.mat[0][coluna + 1] = f"  {coluna +1} "   " "  # cabeçalho
                self.mat[0][0] = "."                # same
                self.mat[linha +1][0] = self.letters[linha] #same
        for c in range(1,len(self.letters)+ 1):
            for a in range(1,len(self.seqs)+ 1):
                self.mat[c][a] = round(float(self.mat[c][a] + pseudo_count + 0.01) / float(len(self.seqs)),3)
        list_temp= []
        list_final = []
        for c in range(1,len(self.seqs)+1):
            for a in range(1,len(self.seqs)+ 1):
                list_temp.append((self.mat[a][c],a))
            list_final.append(max(list_temp))
            list_temp =[]
        
        for c in list_final:
            for a in range(1,len(self.seqs[0])+1):
                if a == c[1]:
                    resul += self.mat[a][0]
        return resul

    

In [ ]:
import unittest  

class Test_Motifs (unittest.TestCase):

    seqs = ['ATTG','ATCG','ATTC','ACTC']


    def test_prob_seq (self):
        self.assertEqual(Motifs(seqs).prob_seq("ATTG"), 0.2845)


    def test_most_prob_seq (self):
        self.assertEqual(Motifs(seqs).most_prob_seq(), "ATTC")

unittest.main(argv=[''], exit=False)
